# 48-redundant-constraints-in-multiproduct
Run models to see what the missing constraint is for a single product

In [1]:
from gpkit import Model, Variable, units
from gpx.multiclass import MClass, MCSystem
from gpx.manufacturing import QNACell, Process, FabLine

In [2]:
import numpy as np

## Create a simple production system with a single product

In [3]:
processes = [Process() for i in range(2)]

In [4]:
processes[0] is processes[1]

False

In [5]:
cells = [QNACell(p) for p in processes]

In [6]:
line = FabLine(cells=cells)

In [7]:
## Update the substitutions
line.substitutions.update({
    # Process Times
    processes[0].t : 15*units('min'),
    processes[1].t : 30*units('min'),
})


In [8]:

### Process CV
line.substitutions.update({
    p.cv : 0.5 for p in processes
})


In [9]:

# Extend the process constraints
line.extend(processes)


Solving fails when extending the model with `processes` more than once

In [10]:

# Constrain the rate
line.substitutions[line.lam] = 50*units('count/hr')

# Set the cost
line.cost = np.sum([c.m for c in cells]) + line.L

In [11]:
print(line)

FabLine

Cost
----
 QNACell.m + QNACell1.m + L

Constraints
-----------
  L = FabLine.\lambda·FabLine.W
  QNACell1.c2a >= QNACell.c2d
  QNACell1.c2d <= QNACell.c2a
  FabLine.\lambda = QNACell.\lambda
  FabLine.\lambda = QNACell1.\lambda
  FabLine.W >= QNACell.W + QNACell1.W

  QNACell
   QNACell.Wq >= QNACell.rho/QNACell.\alpha·(QNACell.c2a + QNACell.c2s)/2·QNACell.t_{\eta}/QNACell.m
   QNACell.c2d >= QNACell.\alpha·QNACell.c2a + QNACell.rho·QNACell.c2s
   QNACell.\alpha + QNACell.rho <= 1
   QNACell.\lambda <= QNACell.rho·QNACell.m/QNACell.t_{\eta}
   QNACell.c2s >= Process1.cv²·QNACell.\chi_{cv}²
   QNACell.W >= QNACell.Wq + QNACell.t_{\eta}
   QNACell.t_{\eta} >= Process1.t·QNACell.\eta_t

  QNACell1
   QNACell1.Wq >= QNACell1.rho/QNACell1.\alpha·(QNACell1.c2a + QNACell1.c2s)/2·QNACell1.t_{\eta}/QNACell1.m
   QNACell1.c2d >= QNACell1.\alpha·QNACell1.c2a + QNACell1.rho·QNACell1.c2s
   QNACell1.\alpha + QNACell1.rho <= 1
   QNACell1.\lambda <= QNACell1.rho·QNACell1.m/QNACell1.t_{\eta}

In [12]:
print(line.solve(solver='cvxopt').table())

Using solver 'cvxopt'
 for 24 free variables
  in 28 posynomial inequalities.
Solving took 0.17 seconds.

Optimal Cost
------------
 83.09

Free Variables
--------------
         | FabLine
       L : 41.35    [count]    Total WIP count
       W : 0.8269   [hr]       Total flow time

         | Process1
  \sigma : 7.5      [min]      standard deviation of process time

         | Process2
  \sigma : 15       [min]      standard deviation of process time

         | QNACell
       W : 16.87    [min]      Total flow time through cell
      Wq : 1.874    [min]      Expected queueing time
  \alpha : 0.1231              (1-rho)
 \lambda : 50       [count/hr] Production rate
     c2a : 0.25                Arrival coefficient of variation squared
     c2d : 0.25                Departure coefficient of variation squared
     c2s : 0.25                Process coefficient of variation squared
       m : 14.25    [count]    Number of parallel workstations
     rho : 0.8769              Cell utiliz

## Smallest model to fail by adding multiple constraints

In [13]:
x = Variable('x')
y = Variable('y')

In [14]:
const = [x >= y]

In [15]:
m = Model(x, const)

In [16]:
m.substitutions[y] = 5

In [17]:
m.solve()

Using solver 'cvxopt'
 for 1 free variables
  in 2 posynomial inequalities.
Solving took 0.00918 seconds.


{'cost': 4.999999919599561,
 'freevariables': {x: 4.999999919579763},
 'constants': {y: 5},
 'variables': {x: 4.999999919579763, y: 5},
 'sensitivities': {'constraints': {gpkit.PosynomialInequality(x >= y): 1.0000000000000004},
  'cost': {x: 1.0},
  'variables': {x: -4.440892098500626e-16, y: 1.0000000000000004},
  'constants': {x: -4.440892098500626e-16, y: 1.0000000000000004}},
 'soltime': 0.009181976318359375}

In [18]:
## Now add the constraint again
m.extend(const)

In [19]:
m

<gpkit.Model object containing 2 top-level constraint(s) and 2 variable(s)>

In [20]:
m.solve()

Using solver 'cvxopt'
 for 1 free variables
  in 3 posynomial inequalities.
Solving took 0.0509 seconds.


{'cost': 5.000003701674993,
 'freevariables': {x: 5.0000036754435495},
 'constants': {y: 5},
 'variables': {x: 5.0000036754435495, y: 5},
 'sensitivities': {'constraints': {gpkit.PosynomialInequality(x >= y): 0.5000000023887884},
  'cost': {x: 1.0},
  'variables': {x: -4.777576667613914e-09, y: 1.0000000047775766},
  'constants': {x: -4.777576667613914e-09, y: 1.0000000047775766}},
 'soltime': 0.05092120170593262}

This does not seem to encounter the same issue when multiple constraints are added as constraints and not as `Model` types

### Adding the multiple constraint as a constraints not as `Model`

## Production System model with added process constraints

In [56]:
line2 = FabLine(cells=cells)

In [57]:
# Substitute times and variation
line2.substitutions.update({
    processes[0].t  : 15*units('min'),
    processes[0].cv : 0.5,
    processes[1].t  : 60*units('min'),
    processes[1].cv : 0.5,
    # rate
    line2.lam : 30*units('count/hr')
})

## Add cost
line2.cost = np.sum([c.m for c in cells]) + line2.L


In [58]:
line2.extend([p.cv == p.stdev/p.t for p in processes])

In [59]:
line2

<gpkit.FabLine object containing 10 top-level constraint(s) and 33 variable(s)>

In [60]:
line2.solve()

Using solver 'cvxopt'
 for 24 free variables
  in 28 posynomial inequalities.
Solving took 0.0757 seconds.


{'cost': 82.77512169795602,
 'freevariables': {QNACell.m: 8.853216361431732,
  QNACell1.m: 32.729075581679325,
  FabLine4.L: 41.19282984571403,
  FabLine4.W: 1.3730943281907306,
  QNACell.c2d: 0.24999999755930044,
  QNACell1.c2a: 0.2499999991564233,
  QNACell1.c2d: 0.25000008648606964,
  QNACell.c2a: 0.2500001702538519,
  QNACell.\lambda: 29.99999999999426,
  QNACell1.\lambda: 29.99999999999426,
  QNACell.W: 17.34760119306645,
  QNACell1.W: 65.03805854132982,
  QNACell.rho: 0.8471497528203272,
  QNACell.\alpha: 0.15285024197842514,
  QNACell.t_{\eta}: 15.000000026826601,
  QNACell.Wq: 2.3476012381727718,
  QNACell.c2s: 0.2500000000133219,
  QNACell1.rho: 0.9166161728453469,
  QNACell1.\alpha: 0.083383827510009,
  QNACell1.t_{\eta}: 60.00000000023803,
  QNACell1.Wq: 5.0380585476734705,
  QNACell1.c2s: 0.25000000407099116,
  Process1.\sigma: 7.49999999999915,
  Process2.\sigma: 29.99999999999426},
 'constants': {FabLine4.\lambda: 30,
  QNACell.\chi_{cv}: 1,
  QNACell.\eta_t: 1,
  Process

This solves successfully. Add the processes again.

In [61]:
# line2.extend([p.cv == p.stdev/p.t for p in processes])
p = processes[0]
line2.extend([p.cv == p.stdev/p.t])
line2.solve()

Using solver 'cvxopt'
 for 24 free variables
  in 30 posynomial inequalities.
Solving took 0.0234 seconds.
The model ran to an infinitely low cost; bounding the right variables would prevent this.
Since this model solved in less than a second, let's run `.debug()` automatically to check.
`
< DEBUGGING >
> Trying with bounded variables and relaxed constants:
>> Failure.
> Trying with relaxed constraints:


IndexError: index 11 is out of bounds for axis 0 with size 11

In [62]:
line2

<gpkit.FabLine object containing 11 top-level constraint(s) and 33 variable(s)>

GPkit struggles using CVXOPT with repeated (redundant) equality constraints!

### Test to see if redundant inequality has the same issue

In [70]:
line3 = FabLine(cells=cells)

In [72]:
# Update substitutions
line3.substitutions.update({
    processes[0].t : 15*units('min'),
    processes[1].t : 30*units('min'),
})

## Process cv
line3.substitutions.update({
    p.cv : 0.5 for p in processes
})

## Production rate
line3.substitutions[line3.lam] = 50*units('count/hr')

# Add Cost
line3.cost = np.sum([c.m for c in cells]) + line3.L

# Add process constraints
line3.extend([
    p.cv == p.stdev/p.t for p in processes
])

In [73]:
line3.solve()

Using solver 'cvxopt'
 for 24 free variables
  in 28 posynomial inequalities.
Solving took 0.0997 seconds.


{'cost': 83.08909000439732,
 'freevariables': {QNACell.m: 14.254327703807236,
  QNACell1.m: 27.489725393424333,
  FabLine6.L: 41.345036920587,
  FabLine6.W: 0.8269007384118443,
  QNACell.c2d: 0.2500000011192617,
  QNACell1.c2a: 0.25000000282944557,
  QNACell1.c2d: 0.25000002282169526,
  QNACell.c2a: 0.25000004512801344,
  QNACell.\lambda: 49.99999999999369,
  QNACell1.\lambda: 49.99999999999369,
  QNACell.W: 16.87449309900153,
  QNACell1.W: 32.73955120895597,
  QNACell.rho: 0.8769266620024282,
  QNACell.\alpha: 0.12307333776052559,
  QNACell.t_{\eta}: 15.000000003108672,
  QNACell.Wq: 1.8744931083368552,
  QNACell.c2s: 0.2500000000097678,
  QNACell1.rho: 0.909430692472707,
  QNACell1.\alpha: 0.09056930764420687,
  QNACell1.t_{\eta}: 30.00000000006062,
  QNACell1.Wq: 2.7395512106665594,
  QNACell1.c2s: 0.2500000008638815,
  Process1.\sigma: 7.499999999999513,
  Process2.\sigma: 14.999999999998687},
 'constants': {FabLine6.\lambda: 50,
  QNACell.\chi_{cv}: 1,
  QNACell.\eta_t: 1,
  Proce

Add a redundant inequality constraint

In [75]:
c = cells[0]

In [76]:
line3.extend([
    c.W >= c.Wq + c.tnu
])

In [77]:
line3

<gpkit.FabLine object containing 11 top-level constraint(s) and 33 variable(s)>

In [78]:
line3.solve()

Using solver 'cvxopt'
 for 24 free variables
  in 29 posynomial inequalities.
Solving took 0.15 seconds.


{'cost': 83.0890899811816,
 'freevariables': {QNACell.m: 14.25432768112975,
  QNACell1.m: 27.489725397344543,
  FabLine6.L: 41.345036911505886,
  FabLine6.W: 0.8269007382303932,
  QNACell.c2d: 0.2500000044431339,
  QNACell1.c2a: 0.25000000886279805,
  QNACell1.c2d: 0.25000000877628964,
  QNACell.c2a: 0.25000001794365256,
  QNACell.\lambda: 49.999999999983345,
  QNACell1.\lambda: 49.999999999983345,
  QNACell.W: 16.87449309264563,
  QNACell1.W: 32.73955121178175,
  QNACell.rho: 0.8769266637043797,
  QNACell.\alpha: 0.12307333565261638,
  QNACell.t_{\eta}: 15.000000005968246,
  QNACell.Wq: 1.8744930797636512,
  QNACell.c2s: 0.25000000000715883,
  QNACell1.rho: 0.9094306923402001,
  QNACell1.\alpha: 0.09056930788469496,
  QNACell1.t_{\eta}: 30.000000000026635,
  QNACell1.Wq: 2.739551216711311,
  QNACell1.c2s: 0.2500000001336316,
  Process1.\sigma: 7.499999999998714,
  Process2.\sigma: 14.999999999996543},
 'constants': {FabLine6.\lambda: 50,
  QNACell.\chi_{cv}: 1,
  QNACell.\eta_t: 1,
  

Seems to work fine with redundant inequality constraints.

## Prep model for creating GPkit Git Issue
Redundant equality constraint causes issues with CVXOPT

When using CVXOPT, a redundant equality constraint will fail to solve and send the model into debug mode (see the example below). The model is feasible as demonstrated by the successful solve on the first go. Once in debug mode, the model is solved successfully with relaxed constraints but shows 0% relaxation.

As far as I have tried, redundant inequality constraints do not seem to have this issue. 

Thoughts?

--Environment Information--

Python 3.
GPkit
CVXOPT

In [86]:
from gpkit import Model, Variable

a = Variable('a', 1)
b = Variable('b')
c = Variable('c', 2)
d = Variable('d')
z = Variable('z', 0.5)

# create a simple GP with equality constraints
const = [
    z == b/a,
    z == d/c,
]

# simple cost
cost = a + b + c + d

# create a model
m = Model(cost, const)

# solve the first version of the model (solves successfully)
m.solve(solver='cvxopt')

# add a redundant equality constraint
m.extend([
    z == b/a
])

# solver will fail and attempt to debug
m.solve(solver='cvxopt')

Using solver 'cvxopt'
 for 2 free variables
  in 5 posynomial inequalities.
Solving took 0.0462 seconds.
Using solver 'cvxopt'
 for 2 free variables
  in 7 posynomial inequalities.
Solving took 0.00747 seconds.
The model ran to an infinitely low cost; bounding the right variables would prevent this.
Since this model solved in less than a second, let's run `.debug()` automatically to check.
`
< DEBUGGING >
> Trying with bounded variables and relaxed constants:
>> Failure.
> Trying with relaxed constraints:

Solves with these constraints relaxed:
   0:    0% relaxed, from z = b/a 
                     to z <= a^-1·b 

>> Success!



{'cost': 4.500001313472465,
 'freevariables': {Relax9.C[:]: array([1., 1., 1.]),
  b: 0.49999999979144816,
  d: 0.9999999984845993},
 'constants': {z: 0.5, a: 1, c: 2},
 'variables': {Relax9.C[:]: array([1., 1., 1.]),
  b: 0.49999999979144816,
  d: 0.9999999984845993,
  z: 0.5,
  a: 1,
  c: 2},
 'sensitivities': {'constraints': {gpkit.PosynomialInequality(Relax9.C[0] >= 1): 10.00081631760442,
   gpkit.PosynomialInequality(Relax9.C[1] >= 1): 10.017579290927342,
   gpkit.PosynomialInequality(Relax9.C[2] >= 1): 10.00081631760442,
   gpkit.PosynomialInequality(z <= Relax9.C[0]·b/a): 10.027369616167594,
   gpkit.PosynomialInequality(Relax9.C[0]·z >= b/a): 9.97181406622798,
   gpkit.PosynomialInequality(z <= Relax9.C[1]·d/c): 10.102321466934313,
   gpkit.PosynomialInequality(Relax9.C[1]·z >= d/c): 9.880099242138336,
   gpkit.PosynomialInequality(z <= Relax9.C[2]·b/a): 10.027369616167594,
   gpkit.PosynomialInequality(Relax9.C[2]·z >= b/a): 9.97181406622798},
  'cost': {Relax9.C[0]: 30.0,
   